# Tool Use

One of the most exciting use cases for LLMs is to use them to provide a natural language interface for other "tools" (whether those be APIs, functions, databases, or otherwise). LangChain is one of the best ways to build this naturl language interface for a few reasons:

- Good output parsing (necessary for returning structured information to pass into a tool)
- Large collection of built-in tools
- Multiple ways of creating the logic to call these tools

In this guide, we will go over two main ways to call tools: chains and agents

## Create a tool

First, we need to create a tool to call. For this example, we will create a custom tool from a function. For more information on all details related to creating custom tools, please see [this guide](/docs/modules/agents/tools/)

In [1]:
from langchain_core.tools import tool


@tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers together."""
    return a * b

In [4]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
multiply(a: int, b: int) -> int - Multiply two integers together.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [5]:
multiply.invoke({"a": 4, "b": 5})

20

## Chains

In chains, you know that you pretty much always want to call a specific tool. Therefore, you don't need to use an LLM to determine IF you want to call a tool, but rather you just need to determine user input.

### Function Calling

One of the most reliable ways to do this is with function calling. This only works models that explicitly support this (like OpenAI).

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.tools.render import format_tool_to_openai_function
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

model = ChatOpenAI(temperature=0)
functions = [format_tool_to_openai_function(multiply)]
model_with_functions = model.bind(functions=functions, function_call={"name": "multiply"})
chain = model_with_functions | JsonOutputFunctionsParser()

In [11]:
chain.invoke("what is thirteen times 4")

{'a': 13, 'b': 4}

If we want to do, we can add instructions in the form of a prompt

In [14]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "Always call multiply with the smaller number as value `a`"),
    ("user", "{input}")
])

chain = prompt | model_with_functions | JsonOutputFunctionsParser()

In [15]:
chain.invoke({"input": "what is thirteen times 4"})

{'a': 4, 'b': 13}

If we want to go further, we can pass the results from the model to the tool

In [18]:
chain_with_tool = chain | multiply

In [19]:
chain_with_tool.invoke({"input": "what is thirteen times 4"})

52

## Agents